In [23]:
import pandas as pd
from math import sqrt

movies_df = pd.read_csv("movies.csv")
ratings_df = pd.read_csv("ratings.csv")


movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))', expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)', expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '', regex=True)
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df = movies_df.drop('genres', axis=1)

ratings_df = ratings_df.drop('timestamp', axis=1)

In [24]:



def get_user_input_dataframe( user_input):
    input_movies = pd.DataFrame(user_input)
    input_id = movies_df[movies_df['title'].isin(input_movies['title'].tolist())]
    input_movies = pd.merge(input_id, input_movies)
    input_movies = input_movies.drop('year', axis=1)
    return input_movies

def filter_user_subset( input_movies):
    user_subset = ratings_df[ratings_df['movieId'].isin(input_movies['movieId'].tolist())]
    return user_subset

def calculate_pearson_correlation( user_subset_group, input_movies):
    pearson_correlation_dict = {}

    for name, group in user_subset_group:
        group = group.sort_values(by='movieId')
        input_movies_subset = input_movies[input_movies['movieId'].isin(group['movieId'].tolist())]

        n_ratings = len(group)
        temp_rating_list = input_movies_subset['rating'].tolist()
        temp_group_list = group['rating'].tolist()

        sxx = sum([i**2 for i in temp_rating_list]) - pow(sum(temp_rating_list), 2) / float(n_ratings)
        syy = sum([i**2 for i in temp_group_list]) - pow(sum(temp_group_list), 2) / float(n_ratings)
        sxy = sum(i*j for i, j in zip(temp_rating_list, temp_group_list)) - sum(temp_rating_list) * sum(temp_group_list) / float(n_ratings)

        if sxx != 0 and syy != 0:
            pearson_correlation_dict[name] = sxy / sqrt(sxx * syy)
        else:
            pearson_correlation_dict[name] = 0

    return pd.DataFrame.from_dict(pearson_correlation_dict, orient='index')





In [25]:
user_input = [
    {'title': 'Avatar', 'rating': 4.5},
    {'title': 'Matrix, The', 'rating':5},
]

In [30]:

input_movies = pd.DataFrame(user_input)
user_subset = ratings_df[ratings_df['movieId'].isin(input_movies['title'].tolist())]
user_subset_group = user_subset.groupby(['userId'])

pearson_df = calculate_pearson_correlation(user_subset_group, input_movies)
pearson_df.columns = ['similarityIndex']

pearson_df['userId'] = pearson_df.index
pearson_df.index = range(len(pearson_df))

print(pearson_df['userId'])

# top_users = pearson_df.sort_values(by='similarityIndex', ascending=False).head(50)



# top_users_rating = top_users.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
# top_users_rating['weightedRating'] = top_users_rating['similarityIndex'] * top_users_rating['rating']

# temp_top_users_rating = top_users_rating.groupby('movieId').sum()[['similarityIndex', 'weightedRating']]
# temp_top_users_rating.columns = ['sum_similarityIndex', 'sum_weightedRating']

# recommendation_df = pd.DataFrame()
# recommendation_df['weighted average recommendation score'] = temp_top_users_rating['sum_weightedRating'] / temp_top_users_rating['sum_similarityIndex']
# recommendation_df['movieId'] = temp_top_users_rating.index

# return recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)


ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [29]:
ratings_df['movieId']

0              169
1             2471
2            48516
3             2571
4           109487
             ...  
22884372     49530
22884373     69481
22884374     74458
22884375     76093
22884376    130636
Name: movieId, Length: 22884377, dtype: int64